# Regularizacion
### Propósito: evitar modelos sobreajustados modificando el comportamiento de descenso por gradiente, objetivo y datos
### Técnica básica: evaluar la bondad de cualquier modificación mediante estimación del rendimiento teórico (en validación)

### 1.1 Vamos a ver las opciones para:  Descenso por Gradiente

#### Idea intuitiva: queremos alcanzar mínimos profundos en regiones anchas, sin caer en mínimos estrechos

##### 1-Terminación temprana: técnica sencilla muy conveniente computacionalmente
##### 2-Learning rate constante: si es algo grande, resultará más difícil caer en mínimos estrechos
##### 3-Planificador del learning rate: quizás con uno o más ciclos de aumento-decremento para evitar mínimos estrechos
##### 4-ReduceLROnPlateau: planificador estándar; caída escalonada monitorizada en validación
##### 5-Dropout: técnica muy efectiva y popular que evita el sobreentrenamiento de neuronas individuales (https://keras.io/api/layers/regularization_layers/dropout)

### 1.2 Vamos a ver las opciones para: Objetivo

#### Batch size: un batch size algo pequeño añade estocasticidad extra al objetivo y dificulta el sobreajuste
#### Penalización de pesos: técnica estándar para penalizar pesos demasiado grandes (en capas seleccionadas)
#### Clase Regularizer: https://keras.io/api/layers/regularizers
#### Tipos de penalizacion: L1, L2 o L1L2 (Elastic net)

### 1.3 Vamos a ver las opciones para: Datos

#### Aumento de datos: el aumento de datos dificulta el sobreajuste (de modelos grandes)
##### Datos sintéticos: en general se obtienen buenos resultados perturbando adecuadamente los de entrenamiento
##### Capas de preproceso de imágenes: https://keras.io/api/layers/preprocessing_layers
##### Capas de aumento de imágenes: https://keras.io/api/layers/preprocessing_layers/image_augmentation


## Ejercicio con Fashion-MNIST

### Inicialización de los datos sin normalizar (regular) 

In [2]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; import keras_tuner
keras.utils.set_random_seed(23); input_dim = (28, 28, 1); num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.astype("float32")
x_test = x_test.astype("float32")
x_train_val = np.expand_dims(x_train_val, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train_val.shape, y_train_val.shape, x_test.shape, y_test.shape)
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]

(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


#### MyHyperModel: exploramos aumento de datos (rotación, translación y zoom) y dropout 0.5

In [13]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        M = keras.Sequential()
        M.add(keras.Input(shape=(28, 28, 1)))
        factor = hp.Float("factor", min_value=0.01, max_value=0.3, step=2, sampling="log")
        M.add(keras.layers.RandomRotation(factor, fill_mode="nearest"))
        M.add(keras.layers.RandomTranslation(factor, factor, fill_mode="nearest"))
        M.add(keras.layers.RandomZoom(factor, fill_mode="nearest"))
        M.add(keras.layers.Rescaling(1./255))
        filters = 64
        M.add(keras.layers.Conv2D(filters, kernel_size=(3, 3), activation="relu"))
        M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        M.add(keras.layers.Conv2D(2*filters, kernel_size=(3, 3), activation="relu"))
        M.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        M.add(keras.layers.Flatten())
        M.add(keras.layers.Dense(units=800, activation='relu'))
        # dropout = hp.Float("dropout", min_value=0.0, max_value=0.5, step=0.1)
        dropout = 0.5
        M.add(keras.layers.Dropout(dropout))
        M.add(keras.layers.Dense(10, activation='softmax'))
        opt = keras.optimizers.Adam(learning_rate=0.00168)
        M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
        return M
    def fit(self, hp, M, x, y, xy_val, **kwargs):
        factor = 0.3787; patience = 5
        reduce_cb = keras.callbacks.ReduceLROnPlateau(
            monitor='val_accuracy', factor=factor, patience=patience, min_delta=1e-4, min_lr=1e-5)
        early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2*patience, min_delta=1e-5)
        kwargs['callbacks'].extend([reduce_cb, early_cb])
        return M.fit(x, y, batch_size=256, epochs=20, validation_data=xy_val, **kwargs)

### Experimento: exploración y resumen de resultados

In [ ]:
tuner = keras_tuner.BayesianOptimization(
MyHyperModel(), objective="val_accuracy", max_trials=10, executions_per_trial=1,
overwrite=True, directory="tmp", project_name="FASHION-MNIST")

In [15]:
tuner.search(x_train, y_train, (x_val, y_val))

Trial 10 Complete [00h 01m 12s]
val_accuracy: 0.9164000153541565

Best val_accuracy So Far: 0.9189000129699707
Total elapsed time: 00h 11m 58s


##### Se sacan los mejores 3 resultados dados en el experimento.

In [16]:
tuner.results_summary(num_trials=3)

Results summary
Results in /tmp/FASHION-MNIST
Showing 3 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
factor: 0.01
Score: 0.9189000129699707

Trial 06 summary
Hyperparameters:
factor: 0.01
Score: 0.9186000227928162

Trial 07 summary
Hyperparameters:
factor: 0.01
Score: 0.9174000024795532


##### Experimento (cont.): evaluación en test de los mejores modelos en validación

In [17]:
num_models = 10
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=num_models)
best_models = tuner.get_best_models(num_models=num_models)
for m in range(num_models):
    values = best_hyperparameters[m].values
    score = best_models[m].evaluate(x_test, y_test, verbose=0)
    print(f'Model {m}: Hyperparameters: {values!s} Loss: {score[0]:.4} Precisión: {score[1]:.2%}')

/home/david/miniconda3/envs/tf217/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model 0: Hyperparameters: {'factor': 0.01} Loss: 0.2918 Precisión: 91.88%
Model 1: Hyperparameters: {'factor': 0.01} Loss: 0.2876 Precisión: 91.46%
Model 2: Hyperparameters: {'factor': 0.01} Loss: 0.3175 Precisión: 91.28%
Model 3: Hyperparameters: {'factor': 0.01} Loss: 0.2916 Precisión: 91.66%
Model 4: Hyperparameters: {'factor': 0.01} Loss: 0.3277 Precisión: 91.25%
Model 5: Hyperparameters: {'factor': 0.01} Loss: 0.2894 Precisión: 90.93%
Model 6: Hyperparameters: {'factor': 0.02} Loss: 0.2749 Precisión: 91.11%
Model 7: Hyperparameters: {'factor': 0.04} Loss: 0.284 Precisión: 90.63%
Model 8: Hyperparameters: {'factor': 0.08} Loss: 0.3048 Precisión: 88.91%
Model 9: Hyperparameters: {'factor': 0.16} Loss: 0.4456 Precisión: 83.69%
